<a href="https://colab.research.google.com/github/alekswheeler/global-_species_abundance_and_diversity/blob/main/AquecimentoGlobalEAbumdanciaDeEspecies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Variáveis utilizadas na análise do artigo *“Estimating Extinction Risks with Species Occurrence Data from the Global Biodiversity Information Facility”* (Dasgupta, Blankespoor e Wheeler, 2024).  

As variáveis analisadas incluem:  

- **Espécies:**  
  - **species:** espécies com dados de ocorrência no GBIF.  
  - **d_assignedrisk:** indicação se a espécie está listada na Lista Vermelha da IUCN (1 para sim, 0 para não).  

- **Ameaças:** fatores que influenciam o risco de extinção, como:  
  - **parkpct:** índice de proteção formal, calculado com base na presença da espécie em áreas protegidas.  
  - **popdens:** densidade populacional, medida em um raster espacial dividido em 15 grupos.  
  - **eezpct:** percentual da área de ocorrência da espécie dentro de Zonas Econômicas Exclusivas (EEZ).  
  - **popshadow:** influência populacional costeira, baseada na projeção de dados terrestres sobre áreas marítimas.  
  - **totfishing:** intensidade total da pesca, estimada por monitoramento via satélite e sistemas AIS.  
  - **natfishing:** intensidade da pesca considerando impactos ambientais, calculada a partir de imagens de satélite.  
  - **pthreat:** índice de probabilidade de ameaça, variando de 0 a 100.  
  - **popgroupmax:** grupo máximo de densidade populacional, classificado em uma escala de 1 a 15.  

O estudo utiliza dados de diversas fontes, incluindo a IUCN, Global Fishing Watch e bases de dados espaciais, para estimar riscos de extinção com base na ocorrência das espécies.

## Definição do problema

## Descrição do conjunto de dados

In [5]:
import os
import zipfile
import requests

# 🔗 URL do Dropbox (modificada para permitir download direto)
dropbox_url = "https://www.dropbox.com/scl/fi/uanikg1a1jklpjyzl5j0x/global_biod_species_extinction_risks.zip?rlkey=fmb0q5n8tth1cvw9rt93a25n4&e=2&st=8iex1p7w&dl=1"

# 📂 Nome do arquivo ZIP localmente no Colab
zip_path = "dataset.zip"

# ⬇️ Baixa o arquivo ZIP do Dropbox
response = requests.get(dropbox_url)
with open(zip_path, "wb") as f:
    f.write(response.content)

print("✅ Download concluído!")

# 📂 Pasta de extração
extract_folder = "/content/dataset_csvs"
os.makedirs(extract_folder, exist_ok=True)

# 🔄 Extrai todos os arquivos CSV do ZIP
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

# 🔍 Lista os arquivos extraídos
csv_files = [f for f in os.listdir(extract_folder) if f.endswith(".csv")]

print("\n📂 Arquivos CSV extraídos:")
for file in csv_files:
    print(f"- {file}")

print("\n✅ Extração concluída! Todos os CSVs estão em:", extract_folder)


✅ Download concluído!

📂 Arquivos CSV extraídos:
- global_biod_species_extinction_risks.csv

✅ Extração concluída! Todos os CSVs estão em: /content/dataset_csvs


In [6]:
import pandas as pd


df = pd.read_csv("/content/dataset_csvs/global_biod_species_extinction_risks.csv")
df

# pega os dados e contém a classe
# https://api.gbif.org/v1/species/match?scientificName=Limonium%20antonii-llorensii

# pode usar o nome que tá na coliuna do CSV. Provavelmente vamos fazer um script para rodar no computador e criar o csv com esses dados.

,species,parkpct,popden,eezpct,popshadow,totfishing,natfishing,popgroupmax,pthreat,d_assignedrisk,lcat
0,01-FULL-36-10b sp001785725,0.111888,NaN,0.681839,519.045361,372.062448,0.043700,6.0,27.0,0.0,Ocean_Coast
1,1-14-0-10-41-10 sp002773695,0.075862,324.278996,0.579310,198.650358,0.404123,0.000000,4.0,54.0,0.0,Coast
2,1-14-0-20-39-34 sp002787355,0.105391,1428.741749,0.603205,800.471147,99.788550,0.000198,6.0,46.0,0.0,Coast
3,1-14-2-50-48-13 sp002783265,0.600000,22.780310,1.000000,90.493047,0.315550,0.000000,4.0,69.0,0.0,Coast
4,12-FULL-67-14b sp016871275,0.097115,NaN,0.668103,444.335993,161.876315,0.021687,6.0,25.0,0.0,Ocean_Coast
...,...,...,...,...,...,...,...,...,...,...,...
597572,Zyzzyx chilensis,0.890100,NaN,NaN,NaN,NaN,NaN,6.0,NaN,0.0,Land_Coast
597573,Zyzzyzus parvula,0.105263,NaN,0.894737,53.343188,6.683700,0.088826,NaN,80.0,0.0,Ocean_Coast
597574,Zyzzyzus rubusidaeus,0.453510,10913.987770,0.880455,4680.094154,32.696057,0.057917,4.0,55.0,0.0,Coast
597575,Zyzzyzus warreni,0.147784,321.172944,0.092674,323.324165,2.677233,0.014074,8.0,20.0,0.0,Coast


In [8]:
pip install pygbif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00


In [44]:
from pygbif import species
# species.name_backbone(name='Helianthus annuus', kingdom='plants')
# species.name_backbone(name='Helianthus', rank='genus', kingdom='plants')
# species.name_backbone(name='Poa', rank='genus', family='Poaceae')

# Verbose - gives back alternatives
# Paging
# species.name_backbone(name='67-14 sp903939845',verbose=False, strict=True, limit=2)
species.name_lookup(q='67-14 sp903939845', verbose=False, limit=1)

{'offset': 0,
 'limit': 1,
 'endOfRecords': False,
 'count': 2,
 'results': [{'key': 183341710,
   'datasetKey': 'a97f36e5-ded1-49cc-bdec-ac6170fc7b9c',
   'nubKey': 11175744,
   'parentKey': 254300782,
   'parent': '67-14',
   'kingdom': 'Bacteria',
   'phylum': 'Actinomycetota',
   'order': 'Solirubrobacterales',
   'family': '70-9',
   'genus': '67-14',
   'species': '67-14 sp903939845',
   'kingdomKey': 254291645,
   'phylumKey': 254300637,
   'classKey': 254300677,
   'orderKey': 254300717,
   'familyKey': 254300754,
   'genusKey': 254300782,
   'speciesKey': 183341710,
   'scientificName': '67-14 sp903939845',
   'nameType': 'OTU',
   'taxonomicStatus': 'ACCEPTED',
   'rank': 'SPECIES',
   'origin': 'SOURCE',
   'numDescendants': 0,
   'numOccurrences': 0,
   'taxonID': 'GB_GCA_903939845.1',
   'habitats': [],
   'nomenclaturalStatus': [],
   'threatStatuses': [],
   'descriptions': [],
   'vernacularNames': [],
   'synonym': False,
   'higherClassificationMap': {'254291645': 'Ba

In [ ]:
import pandas as pd
import requests
import time
from pygbif import species

def get_class_from_api(scientific_name):
    url = f"https://api.gbif.org/v1/species/match?scientificName={scientific_name}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("class", "Unknown")
    return "Unknown"

def get_class_from_pygbif(scientific_name):
    result = species.name_backbone(name=scientific_name, verbose=False, strict=True, limit=2)
    return result.get("class", "Unknown")

def get_extinct_status(scientific_name):
    result = species.name_lookup(q=scientific_name, verbose=False, limit=1)
    if "results" in result and result["results"]:
        return 1 if result["results"][0].get("extinct", False) else 0
    return 0

# Exemplo de DataFrame com a coluna 'species'
data = {"species": ["AG-337-I02 sp902594815", "mammalia"]}
df = pd.DataFrame(data)

# Adicionando a coluna 'class' com os dados da API GBIF e pygbif
df["class"] = df["species"].apply(lambda x: get_class_from_pygbif(x))

# Adicionando a coluna 'extinct'
df["extinct"] = df["species"].apply(lambda x: get_extinct_status(x))

print(df)


# Metodologia

**Todo**
- Adicionar região para filtrar
- Adicionar classe (ou reino, ou filo) [x]
- Adicionar variável Está extinto?
- Encontrar dados da região, dados de clima
- Criar csv co esses dados

**Doing**

**Done**
- Encontrar como adicionar classe (ou reino, ou filo)


## Pré processamento

## Visualização de dados

## Classificação

# Conclusão

## Discussãocação dos Resultados

# Video


Um vídeo (de aproximadamente 5 minutos) descrevendo o trabalho e os resultados. Pretendo divulgar esse vídeo para os demais colegas da disciplina e para os professores do DI. **Os vídeos são apenas para facilitar a correção e não serão publicados!**

# Referências

https://techdocs.gbif.org/en/openapi/

https://datacatalog.worldbank.org/search/dataset/0066034/Global-Biodiversity-Data